## Домашнее задание

В семинаре использовался только небольшой кусочек данных. На всех данных пересчитайте baseline (tfidf). 

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

Нет никаких ограничений кроме:

1) нельзя изменять метрику  
2) решение должно быть воспроизводимым  
3) способы дожны отличаться друг от друга не только гиперпараметрами (например, нельзя три раза поменять гиперпарамтры в TfidfVectorizer и сдать работу)  
4) изменение количества извлекаемых слов не является улучшением (выберите одно значение и используйте только его)  

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://forms.gle/GWzewBEpw8qnkv8t8

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

В поисках идей можно почитать обзоры по теме (посмотрите еще статьи, в которых цитируются эти обзоры): https://www.semanticscholar.org/search?year%5B0%5D=2012&year%5B1%5D=2020&publicationType%5B0%5D=Reviews&q=keyword%20extraction&sort=relevance

**Использовать доступные готовые решения тоже можно**. Так что погуглите перед тем, как приступать к заданию. 

In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import ast

Для обработки решил использовать майстем, т.к. он работате немного быстрее.

Поскольку процесс обработчки всего датасета оказался очень длительным, я сохранил результаты в отдельный файл и далее использовал его. Обработку закомментил.

In [2]:
from pymystem3 import Mystem

In [3]:
m = Mystem()

In [4]:
def is_english(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

Берем только существительные. Если слово написано латиницей, то возвращаем без изменений, т.е. предполагаем, что это термин.

In [5]:
def get_noun(result):
    try:
        if 'analysis' in result.keys():
            if not result['analysis']:
                text = result['text']
                if is_english(text):
                    return text.lower()
                else:
                    return None
            elif result['analysis'][0]['gr'][:2] == 'S,':
                return result['analysis'][0]['lex']
            else:
                return None
    except:
        return None

In [6]:
def normalize(text):
    
    words = m.analyze(text)
    words = [get_noun(word) for word in words]
    words = [word for word in words if word]
    words = [word for word in words if len(word) > 1]

    return words

In [7]:
PATH_TO_DATA = './data'

In [8]:
# files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [9]:
# data = pd.concat([pd.read_json(file, lines=True) for file in files],
#                  axis=0,
#                  ignore_index=True, 
#                  sort=True)

In [10]:
# data.shape

In [11]:
# data.drop(['abstract', 'url'], axis=1, inplace=True)

In [12]:
def to_lower(words):
    return [word.lower() for word in words]

In [13]:
# data['keywords'] = data['keywords'].apply(to_lower)

In [14]:
# %%time

# data['content_norm'] = data['content'].apply(normalize)

In [15]:
# %%time

# data['title_norm'] = data['title'].apply(normalize)

In [16]:
# data.to_csv('normalized.csv', sep='\t', index=False)

Загружаем нормализованные данные из файла для экономии времени.

In [17]:
data = pd.read_csv('normalized.csv', sep='\t')

Пандас не умеет читать списки из напрямую из csv, поэтому преобразуем нужные столбцы.

In [18]:
data['title_norm'] = data['title_norm'].apply(lambda x: ast.literal_eval(x))

In [19]:
data['content_norm'] = data['content_norm'].apply(lambda x: ast.literal_eval(x))

In [20]:
data['keywords'] = data['keywords'].apply(lambda x: ast.literal_eval(x))

In [21]:
data.head()

,content,keywords,summary,title,content_norm,title_norm
0,"Действия России, якобы совершившей кибератаки ...","[выборы в сша, сша, санкции, хакеры, эксклюзив...","В США потребовали осудить Россию за то, что по...",Снова «русские хакеры»: сенат Иллинойса потреб...,"[действие, россия, кибератака, комиссия, штат,...","[хакер, сенат, иллинойс, россия, кибератака]"
1,Президент Латвии Раймонд Вейонис предлагает пр...,"[гражданство, ес, латвия, национализм, права ч...",Президент Латвии Раймонд Вейонис предлагает пр...,Граждане под вопросом: зачем президент Латвии ...,"[президент, латвия, раймонд, вейонис, ребенок,...","[гражданин, вопрос, президент, латвия, паспорт..."
2,Совет Безопасности ООН единогласно принял резо...,"[антониу гутерреш , василий небензя, вооруженн...",Совбез ООН единогласно принял резолюцию 2401 о...,Гуманитарная пауза: как могут развиваться собы...,"[совет, безопасность, оон, резолюция, сторона,...","[пауза, событие, сирия, принятие, резолюция, с..."
3,Выиграть главный футбольный матч клубного сезо...,"[в мире, испания, италия, лига чемпионов уефа,...",Голкипер «Ювентуса» Джанлуиджи Буффон дважды п...,"Попытка номер три, или Последний шанс: Буффон ...","[матч, сезон, европа, одиночка, вратарь, мир, ...","[попытка, номер, шанс, буффон, карьера, лига, ..."
4,Украинским военным стоит быть весьма бдительны...,"[военная техника, киев, корабль, крым, россия,...",В Киеве выразили опасения в связи с возвращени...,«Это уже паранойя»: в России прокомментировали...,"[военный, техника, крым, россия, корабль, пере...","[паранойя, россия, заявление, украина, передач..."


In [22]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [23]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

Если после нормализации ничего не осталось, то сравнивать не имеет смысла.

In [24]:
data = data[data['content_norm_str'] != '']

In [25]:
data.reset_index(inplace=True)

Здесь пришлось задать максимальное количество фич, потому что иначе не хватает памяти.

In [60]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.9, max_features=600)

In [61]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=600, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [62]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [63]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Попробуем подобрать оптимальное количество слов, получаемых данным методом.

In [64]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] #10

In [65]:
evaluate(data['keywords'], keywords)

Precision -  0.07
Recall -  0.08
F1 -  0.07
Jaccard -  0.04


In [66]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] #5

In [67]:
evaluate(data['keywords'], keywords)

Precision -  0.1
Recall -  0.06
F1 -  0.07
Jaccard -  0.04


In [68]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-8:-1]] #7

In [69]:
evaluate(data['keywords'], keywords)

Precision -  0.08
Recall -  0.07
F1 -  0.07
Jaccard -  0.04


In [70]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-16:-1]] #15

In [71]:
evaluate(data['keywords'], keywords)

Precision -  0.05
Recall -  0.1
F1 -  0.07
Jaccard -  0.04


Если берем слишком маленькое или слишком большое, то показатели падают. Для 7 и 10 зеркальные значения точности и полноты. В последующих алгоритмах будем использовать **10** слов.

Таким образом, показатели данного метода на всем датасете оказались ниже, чем были на маленьком кусочке. Итоговый бейзлайн:<br><br>
Precision -  0.07<br>
Recall -  0.08<br>
F1 -  0.07<br>
Jaccard -  0.04

## Способ 1

Попробуем улучшить результат, меняя параметры tfidf.

In [106]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.8, max_features=1000, sublinear_tf=True)

In [107]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=1000, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [108]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [109]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [110]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]]

Удалось получить очень небольшое улучшение в качестве.

In [112]:
evaluate(data['keywords'], keywords)

Precision -  0.08
Recall -  0.1
F1 -  0.08
Jaccard -  0.05


In [113]:
del tfidf
del keywords

## Способ 2

Используем pagerank на всем датасете.

In [114]:
import networkx as nx
from itertools import combinations

Добавил возможность сделать граф направленным.

In [115]:
def build_matrix(text, window_size=5, directed=False):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    ids = [word2id[word] for word in text]

    m = np.zeros((len(vocab), len(vocab)))

    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        for j, k in combinations(window, 2):
            m[j][k] += 1
            if not directed:
                m[k][j] += 1
    
    return m, id2word



In [116]:
def pagerank(text, window_size=5, topn=5, directed=False):
    
    matrix, id2word = build_matrix(text, window_size, directed)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index, measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

Вообще pagerank предполагает все жё направленный граф, поэтому есть смысл сравнивать оба варианта. Результаты на всякий случай сохраняю в отдельной текстовой ячейке, потому что на таком объеме данных алгоритм работает очень долго и не очень удобно каждый раз заново прогонять функцию.

Для сравнения попробовал использовать по 5 и 10 слов в направленных и ненаправленных графах. Используя 5 слов на маленьком кусочке, у направленного графа маленько преимущество.

In [58]:
evaluate(data['keywords'][:100], data['content_norm'][:100].apply(lambda x: pagerank(x, topn=5)))

Precision -  0.25
Recall -  0.14
F1 -  0.17
Jaccard -  0.1


In [59]:
evaluate(data['keywords'][:100], data['content_norm'][:100].apply(lambda x: pagerank(x, topn=5, directed=True)))

Precision -  0.26
Recall -  0.14
F1 -  0.18
Jaccard -  0.1


In [56]:
evaluate(data['keywords'][:100], data['content_norm'][:100].apply(lambda x: pagerank(x, topn=10)))

Precision -  0.19
Recall -  0.21
F1 -  0.19
Jaccard -  0.11


In [57]:
evaluate(data['keywords'][:100], data['content_norm'][:100].apply(lambda x: pagerank(x, topn=10, directed=True)))

Precision -  0.19
Recall -  0.21
F1 -  0.19
Jaccard -  0.11


Для чистоты эксперимента пройдемся по всем данным, выбирая по 10 слов.

In [72]:
keywords_pg = data['content_norm'].apply(lambda x: pagerank(x, topn=10))

In [73]:
evaluate(data['keywords'], keywords_pg)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


Ненаправленный граф, 10 слов.<br><br>
Precision -  0.12 <br>
Recall -  0.17 <br>
F1 -  0.13 <br>
Jaccard -  0.08 <br>

In [74]:
keywords_pg_dir = data['content_norm'].apply(lambda x: pagerank(x, topn=10, directed=True))

In [75]:
evaluate(data['keywords'], keywords_pg_dir)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


Направленный граф, 10 слов. <br><br>
Precision -  0.12 <br>
Recall -  0.17 <br>
F1 -  0.13 <br>
Jaccard -  0.08 <br>

Похоже, что в данном решении направленность роли не играет.

## Способ 3

Попробуем простую меру центральности для сравнения.

In [117]:
def centrality_degree_measure(text, window_size=5, topn=5, directed=False):
    
    matrix, id2word = build_matrix(text, window_size, directed)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.degree_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [120]:
keywords_deg = data['content_norm'].apply(lambda x: centrality_degree_measure(x, topn=10))

In [121]:
evaluate(data['keywords'], keywords_deg)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


Данный алгоритм работает быстро, поэтому можно попробовать разные параметры.

Направленность

In [124]:
keywords_deg = data['content_norm'].apply(lambda x: centrality_degree_measure(x, topn=10, directed=True))

In [125]:
evaluate(data['keywords'], keywords_deg)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


Размер окна

In [122]:
keywords_deg = data['content_norm'].apply(lambda x: centrality_degree_measure(x, 10, 10))

In [198]:
evaluate(data['keywords'], keywords_deg)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


Видим, что в нашей задаче эти параметры ничего не меняют. Получаем тот же результат, но заметно быстрее, чем pagerank.

## Способ 4

Алгоритм и готовая реализация <a href="https://github.com/LIAAD/yake">Yet Another Keyword Extractor (Yake)</a>

In [251]:
import yake

Сразу видно, что выдаваемые результаты действительно похожи на имеющиеся ключевые слова.

In [313]:
language = "ru"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(data.content_norm_str[1])

for kw in keywords:
    print(kw)


('гражданство', 0.0036933995951219712)
('латвия', 0.0038674353118549982)
('негражданин', 0.003870623870057238)
('ребенок', 0.004910473600299539)
('год', 0.006635017495495632)
('человек', 0.007815788396135817)
('государство', 0.008844585219468052)
('язык', 0.009710213770363164)
('страна', 0.009710213770363164)
('право', 0.010620942749700641)


In [344]:
def yake_run(text, topn=10, max_ngram=1):
    language = "ru"
    max_ngram_size = max_ngram
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 2
    numOfKeywords = topn

    custom_kw_extractor = yake.KeywordExtractor(lan=language,
                                                n=max_ngram_size,
                                                dedupLim=deduplication_thresold,
                                                dedupFunc=deduplication_algo,
                                                windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    # The lower the score, the more relevant the keyword is.
    keywords = [kw[0] for kw in keywords]
    return keywords

In [345]:
%%time

evaluate(data.keywords[:100], data.content_norm_str[:100].apply(yake_run))

Precision -  0.19
Recall -  0.2
F1 -  0.19
Jaccard -  0.11
Wall time: 14.7 s


In [343]:
%%time

evaluate(data.keywords[:100], data.content_norm_str[:100].apply(yake_run, max_ngram=2))

Precision -  0.03
Recall -  0.02
F1 -  0.02
Jaccard -  0.01
Wall time: 19 s


Очевидно, что нграмы с длиной больше одного в рамках имеющейся метрики не очень хорошо работают.

In [347]:
keywords_yake = data.content_norm_str.apply(yake_run)

In [348]:
evaluate(data['keywords'], keywords_yake)

Precision -  0.12
Recall -  0.17
F1 -  0.13
Jaccard -  0.08


И опять получаем идентичный результат.

Precision -  0.12 <br>
Recall -  0.17 <br>
F1 -  0.13 <br>
Jaccard -  0.08 <br>